# Exercise - Pricing Swaptions


#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$


# 1. Pricing the Swaption


## Swaption Vol Data

The file `data/swaption_vol_data_2025-06-30.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.


Your data: you will use a single row of this data for the `1x4` swaption.
* date: `2025-06-30`
* expiration: 1yr
* tenor: 4yrs


## Rate Data

The file `data/cap_curves_2025-06-30.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.


## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.


### 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.


In [1]:
import re

import numpy as np
import pandas as pd
from scipy.stats import norm


def tenor_to_years(x) -> float:
    """Parse tenor values into years (supports numbers, '1y/1yr', '12m', '3mo', etc.)."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)

    s = str(x).strip().lower().replace(" ", "")
    s = (
        s.replace("yrs", "y")
        .replace("yr", "y")
        .replace("years", "y")
        .replace("year", "y")
        .replace("mos", "m")
        .replace("mo", "m")
        .replace("months", "m")
        .replace("month", "m")
    )

    m = re.fullmatch(r"([0-9]*\.?[0-9]+)([a-z]*)", s)
    if not m:
        raise ValueError(f"Cannot parse tenor: {x!r}")

    val = float(m.group(1))
    unit = m.group(2)

    scale = {"": 1.0, "y": 1.0, "m": 1.0 / 12.0}
    if unit in scale:
        return val * scale[unit]

    if unit in {"w", "wk", "wks", "week", "weeks"}:
        return val / 52.0

    raise ValueError(f"Unrecognized tenor unit in {x!r}")


def pick_col(df: pd.DataFrame, *, candidates: list[str]) -> str:
    """Pick the first column whose normalized name contains any candidate substring."""
    normed = {c: str(c).strip().lower().replace(" ", "") for c in df.columns}
    for cand in candidates:
        cand_norm = cand.strip().lower().replace(" ", "")
        hit = next((c for c, n in normed.items() if cand_norm in n), None)
        if hit is not None:
            return hit
    raise KeyError(f"None of {candidates} found in columns: {list(df.columns)}")


def df_at_from_series(dfs: pd.Series, t: float) -> float:
    """Discount factor P(0,t) with log-linear interpolation in DF space."""
    if t in dfs.index:
        return float(dfs.loc[t])

    x = dfs.index.to_numpy(dtype=float)
    y = np.log(dfs.to_numpy(dtype=float))
    return float(np.exp(np.interp(t, x, y)))


def black_payer_swaption_price(*, fwd: float, strike: float, vol: float, expiry: float, annuity: float, notional: float = 1.0) -> float:
    """Black payer swaption PV at t=0: N * A * (F N(d1) - K N(d2))."""
    if expiry <= 0 or vol <= 0:
        return float(notional * annuity * max(fwd - strike, 0.0))
    if fwd <= 0 or strike <= 0:
        raise ValueError(f"Black formula requires positive fwd/strike. Got fwd={fwd}, strike={strike}")

    vol_sqrt_t = float(vol * np.sqrt(expiry))
    d1 = (np.log(fwd / strike) + 0.5 * vol * vol * expiry) / vol_sqrt_t
    d2 = d1 - vol_sqrt_t
    return float(notional * annuity * (fwd * norm.cdf(d1) - strike * norm.cdf(d2)))


In [2]:
# Rate/DF data (single-curve setup for this exercise)
curve = (
    pd.read_excel(
        "cap_curves_2025-06-30.xlsx",
        sheet_name="rate curves 2025-06-30",
    )
    .set_index("tenor")
    .sort_index()
)

dfs = curve["discounts"].astype(float)  # P(0,T)


# 1y forward, 4y underlying swap => swap runs from T0=1.0 to Tn=5.0
T0 = 1.00
Tn = 5.00
pay_freq = 4  # quarterly
alpha = 1.0 / pay_freq

pay_times = np.arange(T0 + alpha, Tn + 1e-12, alpha)  # 1.25, 1.50, ..., 5.00

P0_T0 = df_at_from_series(dfs, T0)
P0_Tn = df_at_from_series(dfs, Tn)

# Fixed-leg annuity A(0) = sum_i alpha_i * P(0, T_i)
annuity = float(np.sum(alpha * np.array([df_at_from_series(dfs, t) for t in pay_times], dtype=float)))

# Forward par swap rate S_fwd(0; T0,Tn)
S_fwd = (P0_T0 - P0_Tn) / annuity

print(f"P(0,{T0:.2f}) = {P0_T0:.6f}")
print(f"P(0,{Tn:.2f}) = {P0_Tn:.6f}")
print(f"Annuity A(0)   = {annuity:.6f}")
print(f"1y fwd 4y swap rate (quarterly) = {S_fwd:.6%}")


P(0,1.00) = 0.962807
P(0,5.00) = 0.844989
Annuity A(0)   = 3.603238
1y fwd 4y swap rate (quarterly) = 3.269770%




### 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.


In [3]:
# --- Load swaption vol skew for 1y x 4y ---
vol_file = "swaption_vol_data_2025-06-30.xlsx"
xls = pd.ExcelFile(vol_file)
vol_sheet = xls.sheet_names[0]
vols = pd.read_excel(vol_file, sheet_name=vol_sheet)

expry_col = pick_col(vols, candidates=["expry", "expiry", "expiration", "exp"])
tenor_col = pick_col(vols, candidates=["tenor", "term", "swaptenor"])
try:
    model_col = pick_col(vols, candidates=["model", "quote", "voltype"])
except KeyError:
    model_col = None

vols = vols.copy()
vols["expry_years"] = vols[expry_col].apply(tenor_to_years)
vols["tenor_years"] = vols[tenor_col].apply(tenor_to_years)

mask = (vols["expry_years"] == 1.0) & (vols["tenor_years"] == 4.0)
target = vols[mask]
if model_col is not None:
    target = target[target[model_col].astype(str).str.lower().str.contains("black")]

if len(target) == 0:
    raise ValueError(
        "Could not find a 1y x 4y swaption row in the vol file. "
        f"Found expry_col={expry_col!r}, tenor_col={tenor_col!r}, model_col={model_col!r}."
    )
row = target.iloc[0]

# strike offsets in bps live in columns named like -200, -100, 0, 100, ...
offset_cols = sorted(
    {
        int(float(str(c).strip()))
        for c in vols.columns
        if re.fullmatch(r"-?\d+(?:\.0+)?", str(c).strip())
    }
)

expiry = float(row["expry_years"])
fwd = float(S_fwd)
A0 = float(annuity)
notional = 1.0

results = []
for bps in offset_cols:
    key_candidates = (bps, str(bps), float(bps))
    v = next((row[k] for k in key_candidates if k in row.index), np.nan)
    if pd.isna(v):
        continue

    vol = float(v)
    vol = vol / 100.0 if vol > 3.0 else vol

    strike = fwd + bps / 10_000.0
    pv = black_payer_swaption_price(fwd=fwd, strike=strike, vol=vol, expiry=expiry, annuity=A0, notional=notional)

    results.append(
        {
            "offset_bps": bps,
            "strike": strike,
            "imp_vol": vol,
            "pv_per_notional": pv,
            "pv_bps_of_notional": pv * 10_000.0,
        }
    )

out = pd.DataFrame(results).sort_values("offset_bps").reset_index(drop=True)

print(f"Using vol sheet: {vol_sheet!r}")
print(f"Columns used: expry={expry_col!r}, tenor={tenor_col!r}, model={model_col!r}")
print(f"Forward swap rate F (1y fwd 4y): {fwd:.6%}")
print(f"Annuity A(0): {A0:.6f}")
print(f"Expiry T: {expiry:.2f}y")

display(out.style.format({"strike": "{:.6%}", "imp_vol": "{:.2%}", "pv_per_notional": "{:.6f}", "pv_bps_of_notional": "{:.2f}"}))


Using vol sheet: 'bloomberg vcub'
Columns used: expry='expiration', tenor='tenor', model='model'
Forward swap rate F (1y fwd 4y): 3.269770%
Annuity A(0): 3.603238
Expiry T: 1.00y


,offset_bps,strike,imp_vol,pv_per_notional,pv_bps_of_notional
0,-200,1.269770%,54.41%,0.072711,727.11
1,-100,2.269770%,38.56%,0.039480,394.80
2,-50,2.769770%,33.92%,0.025362,253.62
3,-25,3.019770%,32.20%,0.019431,194.31
4,0,3.269770%,30.83%,0.014434,144.34
5,25,3.519770%,29.80%,0.010424,104.24
6,50,3.769770%,29.09%,0.007366,73.66
7,100,4.269770%,28.43%,0.003557,35.57
8,200,5.269770%,28.88%,0.000880,8.80



### 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the swaption above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.)
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.


In [4]:
# Use the ATM implied vol from the 1y x 4y skew for all comparisons (as requested in 1.3)
atm_row = out.loc[out["offset_bps"] == 0]
if len(atm_row) != 1:
    raise ValueError("Could not uniquely identify ATM vol at offset_bps=0.")

atm_vol = float(atm_row["imp_vol"].iloc[0])


def fwd_swap_rate_and_annuity(*, dfs: pd.Series, start: float, end: float, pay_freq: int) -> tuple[float, float]:
    """Return (forward par swap rate, fixed-leg annuity) for swap from start to end."""
    alpha = 1.0 / pay_freq
    pay_times = np.arange(start + alpha, end + 1e-12, alpha)

    p_start = df_at_from_series(dfs, start)
    p_end = df_at_from_series(dfs, end)

    ann = float(np.sum(alpha * np.array([df_at_from_series(dfs, t) for t in pay_times], dtype=float)))
    fwd = (p_start - p_end) / ann
    return float(fwd), float(ann)


swaptions = [
    {"name": "1y x 4y", "expiry": 1.00, "tenor": 4.00},
    {"name": "3mo x 4y", "expiry": 0.25, "tenor": 4.00},
    {"name": "2y x 4y", "expiry": 2.00, "tenor": 4.00},
    {"name": "1y x 2y", "expiry": 1.00, "tenor": 2.00},
]

rows = []
for s in swaptions:
    start = float(s["expiry"])
    end = float(s["expiry"] + s["tenor"])

    fwd_i, ann_i = fwd_swap_rate_and_annuity(dfs=dfs, start=start, end=end, pay_freq=pay_freq)
    strike_i = fwd_i  # ATM

    pv_i = black_payer_swaption_price(
        fwd=fwd_i,
        strike=strike_i,
        vol=atm_vol,
        expiry=start,
        annuity=ann_i,
        notional=1.0,
    )

    rows.append(
        {
            "swaption": s["name"],
            "expiry_y": start,
            "tenor_y": float(s["tenor"]),
            "swap_start_y": start,
            "swap_end_y": end,
            "pay_freq": pay_freq,
            "atm_vol_used": atm_vol,
            "fwd_swap_rate": fwd_i,
            "strike": strike_i,
            "annuity": ann_i,
            "pv_per_notional": pv_i,
            "pv_bps_of_notional": pv_i * 10_000.0,
        }
    )

summary = pd.DataFrame(rows)

print(f"ATM vol used for all cases (from 1y x 4y, offset=0): {atm_vol:.2%}")
display(
    summary.style.format(
        {
            "expiry_y": "{:.2f}",
            "tenor_y": "{:.2f}",
            "swap_start_y": "{:.2f}",
            "swap_end_y": "{:.2f}",
            "atm_vol_used": "{:.2%}",
            "fwd_swap_rate": "{:.6%}",
            "strike": "{:.6%}",
            "annuity": "{:.6f}",
            "pv_per_notional": "{:.6f}",
            "pv_bps_of_notional": "{:.2f}",
        }
    )
)


ATM vol used for all cases (from 1y x 4y, offset=0): 30.83%


,swaption,expiry_y,tenor_y,swap_start_y,swap_end_y,pay_freq,atm_vol_used,fwd_swap_rate,strike,annuity,pv_per_notional,pv_bps_of_notional
0,1y x 4y,1.00,4.00,1.00,5.00,4,30.83%,3.269770%,3.269770%,3.603238,0.014434,144.34
1,3mo x 4y,0.25,4.00,0.25,4.25,4,30.83%,3.299759%,3.299759%,3.692194,0.007485,74.85
2,2y x 4y,2.00,4.00,2.00,6.00,4,30.83%,3.425671%,3.425671%,3.484495,0.020599,205.99
3,1y x 2y,1.00,2.00,1.00,3.00,4,30.83%,3.118344%,3.118344%,1.860504,0.007108,71.08


Generally the value of swaptions gets bigger along with the growth of expiry and tenor.